In [1]:
import joblib 
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm
import librosa
import glob
from keras.utils import np_utils
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Activation, MaxPooling1D, Flatten, BatchNormalization, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss, classification_report
from tensorflow.keras import backend
import tensorflow_addons as tfa
from tensorflow.keras.metrics import Recall

In [2]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [3]:
DIR= './input/ravdess'

In [4]:
def extract_features(file_name, mfcc, chroma, mel):
    X, sample_rate= librosa.load(os.path.join(file_name), res_type= 'kaiser_fast')
    if chroma:
        stft= np.abs(librosa.stft(X))
    result= np.array([])
    if mfcc:
        mfcc= np.mean(librosa.feature.mfcc(y=X, sr= sample_rate, n_mfcc= 40).T, axis=0)
        result= np.hstack((result, mfcc))
    if chroma:
        chroma= np.mean(librosa.feature.chroma_stft(S= stft, sr= sample_rate).T, axis=0)
        result= np.hstack((result, chroma))
    if mel:
        mel= np.mean(librosa.feature.melspectrogram(y=X, sr= sample_rate).T, axis=0)
        result= np.hstack((result, mel))
    return result

In [5]:
def test_data():
    x= []

    file1= '../input/ravdess/help me.wav'
    file2= '../input/ravdess/t1.wav'
    features= extract_features(file1, mfcc= True, chroma= True, mel= True)
    x.append(features)
    features= extract_features(file2, mfcc= True, chroma= True, mel= True)
    x.append(features)
    return x

In [6]:
test= np.array(test_data())
testcnn= np.expand_dims(test, axis=2)

In [7]:
features_DIR= '../input/ravdess/acoustic_features.joblib'
labels_DIR= '../input/ravdess/labels.joblib'
x, y = joblib.load(features_DIR), joblib.load(labels_DIR)
x= np.array(x)
le= LabelEncoder()
y= np_utils.to_categorical(le.fit_transform(y))
# ohe= OneHotEncoder()
# y= ohe.fit_transform(y.reshape(-1,1))
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.25,  random_state= 42, shuffle= True)

In [8]:
x_train.shape

(1500, 180)

In [9]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
MPLC =MLPClassifier(alpha=0.001, 
                    batch_size=256, 
                    epsilon=1e-08, 
                    hidden_layer_sizes=(300,), 
                    learning_rate='adaptive', 
                    max_iter=1000)

In [10]:
MPLC.fit(x_train, y_train)

MLPClassifier(alpha=0.001, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=1000)

In [11]:
joblib.dump(MPLC, 'MPLC.h5')
MPLC.score(x_test, y_test)

0.646

In [12]:
list(le.classes_)

['female_angry',
 'female_calm',
 'female_fearful',
 'female_happy',
 'female_sad',
 'male_angry',
 'male_calm',
 'male_fearful',
 'male_happy',
 'male_sad']

In [13]:
b= MPLC.predict_proba(test)

In [14]:
classification_report(y_test, MPLC.predict(x_test))

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0       0.96      0.86      0.91        79\n           1       0.54      0.64      0.58        22\n           2       0.95      0.81      0.87        73\n           3       0.95      0.91      0.93        82\n           4       0.90      0.78      0.84        72\n           5       0.52      0.43      0.47        35\n           6       1.00      0.33      0.50        30\n           7       0.60      0.33      0.43        36\n           8       0.77      0.45      0.57        38\n           9       0.71      0.30      0.43        33\n\n   micro avg       0.85      0.67      0.75       500\n   macro avg       0.79      0.58      0.65       500\nweighted avg       0.84      0.67      0.73       500\n samples avg       0.66      0.67      0.66       500\n'

In [15]:
svc= SVC()
svc.probability=True

In [16]:
svc.fit(x_train, np.argmax(y_train, axis= 1))

SVC(probability=True)

In [17]:
svc.score(x_test, np.argmax(y_test, axis=1))

0.556

In [18]:
a= svc.predict_proba(test)

In [19]:
joblib.dump(svc, 'svc.h5')

['svc.h5']

In [20]:
c= a+b

In [21]:
np.argmax(c, axis=1)

array([8, 2])